# 对点云将采样后对齐叠加

In [25]:
import pykitti
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
basedir = '../datasets'
date = '2011_10_03'
drive = '0027'
dataset = pykitti.raw(basedir, date, drive)

# 对前20帧降采样

In [26]:
point_cloud = o3d.geometry.PointCloud()
voxel_size = 2.0
dataset_downsampled = []
#num_frames = len(dataset) 总帧数
num_frames = 200
for idx in range(num_frames):
    data = dataset.get_velo(idx)
    point_cloud.points = o3d.utility.Vector3dVector(data[:, :3])
    color = np.random.uniform(0, 1, size=3)
    point_cloud_downsampled = point_cloud.voxel_down_sample(voxel_size)
    point_cloud_downsampled.paint_uniform_color(color)
    dataset_downsampled.append(point_cloud_downsampled)

dataset_downsampled = np.asarray(dataset_downsampled)
dataset_downsampled.shape
#简单可视化验证
#o3d.visualization.draw_geometries([dataset_downsampled[0]])
#o3d.visualization.draw_geometries([dataset_downsampled[19]])



(200,)

# 匹配+叠加

In [27]:
# 初始20帧点云展示（重叠，雷达自身坐标系）
o3d.visualization.draw_geometries(dataset_downsampled)

读取GPS信息的经纬度与高度，并转换为笛卡尔坐标系
提取前六个值 

lat: OXTS单元的纬度（度）

lon: OXTS单元的经度（度）

alt: OXTS单元的高度（米）

roll: 横滚角（弧度），0 = 水平，正值 = 左侧抬起（-π..π）

pitch: 俯仰角（弧度），0 = 水平，正值 = 前部向下（-π/2..π/2）

yaw: 航向（弧度），0 = 东，正值 = 逆时针（-π..π）

转换为笛卡尔坐标系：使用以下公式将相对经纬度和高度转换为XYZ坐标：

X=(lon−lonref)×cos(latref)×R

Y=(lat−latref)×R

Z=alt−altref

In [28]:
import os
import math

def read_oxts_data(directory):
    data_list = []
    for filename in sorted(os.listdir(directory)):
        file_path = os.path.join(directory, filename)
        with open(file_path, 'r') as file:
            line = file.readline().strip()
            values = line.split()[:6]  
            data_list.append([float(val) for val in values])  # 转换为float并添加到列表
    return data_list

# 使用函数读取数据
directory = '../datasets/2011_10_03/2011_10_03_drive_0027_sync/oxts/data'  
oxts_data = read_oxts_data(directory)

Cartesian_data = []
lat0 = oxts_data[0][0]
lon0 = oxts_data[0][1]
alt0 = oxts_data[0][2]
R = 6371000 #地球半径6731公里
for idx in range(len(oxts_data)):
    lat = oxts_data[idx][0]
    lon = oxts_data[idx][1]
    alt = oxts_data[idx][2]
    roll = oxts_data[idx][3]
    pitch = oxts_data[idx][4]
    yaw = oxts_data[idx][5]
    x = (lon - lon0) * R * math.cos(lat0)
    y = (lat - lat0) * R
    z = (alt - alt0)
    Cartesian_data.append([x, y, z, roll, pitch, yaw])

print(Cartesian_data[0])
print(Cartesian_data[1])
print(Cartesian_data[2])

[0.0, 0.0, 0.0, 0.042203, 0.022275, 1.0395963267949]
[9.492670814415975, 42.094917184236635, 0.0008583068849929987, 0.041757, 0.0210905, 1.0415528267949]
[18.985341632044896, 84.1898343232046, 0.0017166137700002082, 0.041311, 0.019906, 1.0435093267949]


将点云转换到global坐标系下

In [29]:
import open3d as o3d
import numpy as np

def create_rotation_matrix(roll, pitch, yaw):
    R_x = np.array([[1, 0, 0],
                    [0, np.cos(roll), -np.sin(roll)],
                    [0, np.sin(roll), np.cos(roll)]])

    R_y = np.array([[np.cos(pitch), 0, np.sin(pitch)],
                    [0, 1, 0],
                    [-np.sin(pitch), 0, np.cos(pitch)]])

    R_z = np.array([[np.cos(yaw), -np.sin(yaw), 0],
                    [np.sin(yaw), np.cos(yaw), 0],
                    [0, 0, 1]])

    return np.dot(R_z, np.dot(R_y, R_x))

def transform_pointcloud(point_cloud, x, y, z, roll, pitch, yaw):
    # 创建平移矩阵
    translation = np.array([x, y, z])
    T = np.eye(4)
    T[:3, 3] = translation

    # 创建旋转矩阵
    R = create_rotation_matrix(roll, pitch, yaw)
    R_homogeneous = np.eye(4)
    R_homogeneous[:3, :3] = R

    # 组合变换矩阵
    transformation_matrix = np.dot(T, R_homogeneous)

    # 应用变换
    point_cloud = point_cloud.transform(transformation_matrix)
    
    return point_cloud

In [30]:
import open3d as o3d
point_cloud_transformed = []
for i in range(num_frames):
    point_cloud = dataset_downsampled[i]
    x, y, z = Cartesian_data[i][0], Cartesian_data[i][1], Cartesian_data[i][2]
    roll, pitch, yaw = Cartesian_data[i][3], Cartesian_data[i][4], Cartesian_data[i][5]
    point_cloud = transform_pointcloud(point_cloud, x, y, z, roll, pitch, yaw)
    point_cloud_transformed.append(point_cloud)

# 可视化
o3d.visualization.draw_geometries(point_cloud_transformed)